In [69]:
import requests
import pandas as pd
import yaml
from yaml import BaseLoader
from tqdm import tqdm
import json

# Load the YAML file.
with open("key.yaml") as f:
    key = yaml.load(f, Loader=BaseLoader)

In [70]:
df = pd.read_csv('./data/lasso-sneaks.csv')
shoes = list(df['Shoe'].unique())
shoes

['Nike Air Pegasus 92 Lite',
 'Nike Tanjun',
 'Jordan Zoom Zero Gravity',
 'Jordan React Havoc Paris Saint-Germain (PSG)',
 'Nike Tanjun in Midnight Navy/Game Royal/White',
 'Nike Air Huarache Premium ‘Varsity Jacket’ in Red',
 'Nike Air Zoom Pegasus\xa0Lite Total Orange\xa092',
 'Odyssey React 2 Flyknit ‘Black Racer Blue’',
 'Eric Koston Air Jordan 1 Low ‘Powder Blue’',
 'JIMMY CHOO Diamond X Strap/F Leather Sneakers',
 'Nike Blazer Low X ‘White Light Photo Blue’',
 'Eric Koston x Air Jordan 1 Low SB ‘Powder Blue’',
 'Nike Waffle One Summer of Sports Pack',
 'ASIC Mexico 66 ‘Kill Bill’',
 'Nike Air Jordan 3 ‘Muslin’',
 'Red Nike Daybreak Type ‘Red Racer Blue’',
 'Nike Daybreak ‘Light Armory Blue’',
 'Nike Air Jordan 5 “Chinese New Year”',
 'Nike Air Jordan 1 Low Light Smoke Grey',
 'Nike Air Jordan 1 Low ‘White Camo’',
 'Air Jordan 13 “Lucky Green”',
 'Nike Phantom Vision Academy Indoor Turf Soccer Shoes',
 'New Balance 515 Sport V2 White Pigment Red ML515OTX',
 'New Balance 515 V2 Wh

In [71]:
import re
import time

url = "https://sneaker-database-stockx.p.rapidapi.com/getproducts"

headers = {
	"X-RapidAPI-Key": key['RAPID_API_KEY'],
	"X-RapidAPI-Host": "sneaker-database-stockx.p.rapidapi.com"
}

sneaker_dict = {}
for i, sneaker in enumerate(shoes): 
    
	try:
		# cleaning up quote characters
		sneaker_str = re.sub(r'[^A-Za-z0-9 ]+', ' ', sneaker)
		sneaker_str = re.sub(' +', ' ', sneaker_str).strip()
		sneaker_str = sneaker_str.replace(" Light ", " ")
		sneaker_str = sneaker_str.replace(" X Strap F Leather ", " ")

		# make API request to the getProducts endpoint of the Sneaker Database - StockX API, save response in dict
		querystring = {"keywords":sneaker_str,"limit":"1"}
		response = requests.get(url, headers=headers, params=querystring)
		sneaker_dict[sneaker] = json.loads(response.text)[0]
		print(i, f"SUCCESS: {sneaker}")

	except Exception as e:
		print(i, f"ERROR: {sneaker} ({sneaker_str})")
		print(e)

	# wait 2 minutes just in case we hit the rate limit
	time.sleep(120)


0 SUCCESS: Nike Air Pegasus 92 Lite
1 SUCCESS: Nike Tanjun
2 SUCCESS: Jordan Zoom Zero Gravity
3 SUCCESS: Jordan React Havoc Paris Saint-Germain (PSG)
4 SUCCESS: Nike Tanjun in Midnight Navy/Game Royal/White
5 SUCCESS: Nike Air Huarache Premium ‘Varsity Jacket’ in Red
6 SUCCESS: Nike Air Zoom Pegasus Lite Total Orange 92
7 SUCCESS: Odyssey React 2 Flyknit ‘Black Racer Blue’
8 SUCCESS: Eric Koston Air Jordan 1 Low ‘Powder Blue’
9 SUCCESS: JIMMY CHOO Diamond X Strap/F Leather Sneakers
10 SUCCESS: Nike Blazer Low X ‘White Light Photo Blue’
11 SUCCESS: Eric Koston x Air Jordan 1 Low SB ‘Powder Blue’
12 SUCCESS: Nike Waffle One Summer of Sports Pack
13 SUCCESS: ASIC Mexico 66 ‘Kill Bill’
14 SUCCESS: Nike Air Jordan 3 ‘Muslin’
15 ERROR: Red Nike Daybreak Type ‘Red Racer Blue’ (Red Nike Daybreak Type Red Racer Blue)
'NoneType' object is not subscriptable
16 SUCCESS: Nike Daybreak ‘Light Armory Blue’
17 SUCCESS: Nike Air Jordan 5 “Chinese New Year”
18 SUCCESS: Nike Air Jordan 1 Low Light Smoke

In [76]:
with open("./data/sneaker_info.json", "w") as outfile:
    json.dump(sneaker_dict, outfile)